In [1]:
# import library
import pickle, os
import pandas as pd
import numpy as np
import tqdm.notebook as tqdm

In [ ]:
pip install sirius
데이터 다운로더 자체를 내장시키는 방법도 가능
pip install Moggle_datadownloader  (셀레니움으로 KRX데이터를 다운로드 / fundamental data 다운로드)
config 파일 생성 ->  directory

sirius를 가지고 API 같은 느낌이 나도록? : REST API로 구현이 되어 있는데 이걸 아예 sirius 안에 내장시켜도 좋을듯!

In [ ]:
홈페이지를 재정비

In [2]:
# Data Download
with open('FS_손익계산서보정본', 'rb') as f:
    data = pickle.load(f)

with open('Final_FS_ind', 'rb') as f:
    ind = pickle.load(f)

In [3]:
# 특정 자산만 뽑아서 csv로 정리하기
def get_item(concept_id):
    item = []
    item_ind = []
    for i in range(len(data)):
        if type(data[i][0]) == type(data[0][0]):
            item.append(data[i][0][data[i][0][data[i][0].columns[0]] == concept_id])
            if len(data[i][0][data[i][0][data[i][0].columns[0]] == concept_id]) > 0:
                item_ind.append(ind[i])
    df_item = pd.concat(item)
    date = list(filter(lambda x: x[1] == ('연결재무제표',), df_item.columns))
    
    date_clean = list(map(lambda x: x[0], df_item[date].columns))
    df_item[date].columns = date_clean
    df_item2 = df_item[date]
    df_item2.columns = date_clean
    date_shifted = list(map(lambda x: shift_q(x), date_clean))
    df_item3 = df_item2
    df_item3.columns = date_shifted
    df_item4 = df_item3.groupby(level=0, axis=1).last()
    df_item3 = df_item4
    date_shifted2 = list(map(lambda x: shift_d(x), df_item4.columns))
    df_item3.columns = date_shifted2
    df_item5 = df_item3.groupby(level=0, axis=1).last()
    corp_code = list(map(lambda x:x[2], item_ind))
    df_item5['corp_code'] = corp_code
    df_item5 = df_item5.set_index('corp_code')
    return df_item5

def shift_q(date):
        if date[4:6]=='01' or date[4:6]=='02':
            date = date[:4] + '0331'
        elif date[4:6]=='04' or date[4:6]=='05':
            date = date[:4] + '0630'
        elif date[4:6]=='07' or date[4:6]=='08':
            date = date[:4] + '0930'
        elif date[4:6]=='10' or date[4:6]=='11':
            date = date[:4] + '1231'
        return date

def shift_d(date):
    # 케이스 두개 밖에 안되니까 그냥 수동으로 따져주자.
    if date[6:]=='01':
        date = date[:6] + '31'
    elif date[6:]=='29':
        date = date[:6] + '30'
    return date

In [381]:
get_item('ifrs_Equity')

C:\Users\lyoo9\anaconda3\lib\site-packages\pandas\core\dtypes\cast.py:1229: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  v = np.array(v, copy=False)
C:\Users\lyoo9\anaconda3\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


,20091231,20100331,20100630,20100930,20101231,20110331,20110630,20110930,20111231,20120331,...,20190331,20190630,20190930,20191231,20200331,20200630,20200930,20201231,20210331,20210630
corp_code,,,,,,,,,,,,,,,,,,,,,
019660,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
024840,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,8.476296e+10,8.635589e+10,8.781197e+10,9.330697e+10,9.251335e+10,9.685097e+10,9.963772e+10,9.594783e+10,9.570079e+10,NaN
067830,2.270192e+11,NaN,NaN,NaN,2.366435e+11,NaN,NaN,NaN,2.508800e+11,NaN,...,4.392618e+11,4.400014e+11,4.433818e+11,4.479321e+11,4.491193e+11,4.553963e+11,4.553342e+11,NaN,NaN,NaN
101170,5.828420e+10,NaN,NaN,NaN,6.498524e+10,NaN,NaN,NaN,7.284740e+10,NaN,...,NaN,NaN,NaN,8.678080e+10,8.793617e+10,8.816953e+10,8.834150e+10,7.918151e+10,7.877699e+10,NaN
149950,NaN,NaN,NaN,NaN,2.762930e+10,NaN,NaN,NaN,5.119392e+10,NaN,...,NaN,NaN,NaN,1.360878e+11,NaN,1.350568e+11,1.380093e+11,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
010140,3.057477e+12,NaN,NaN,NaN,4.132455e+12,NaN,NaN,NaN,4.643989e+12,4.790098e+12,...,6.667831e+12,6.352703e+12,5.839378e+12,5.248929e+12,5.273925e+12,4.522799e+12,4.428057e+12,3.718232e+12,3.336425e+12,2.907945e+12
096530,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.312505e+11,1.410191e+11,1.469833e+11,1.528257e+11,1.841244e+11,3.098118e+11,4.615711e+11,6.464059e+11,7.709263e+11,8.718612e+11
225190,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.104104e+10,3.427206e+10,3.383998e+10,3.928479e+10,3.392776e+10,3.423793e+10,3.400208e+10,3.451267e+10,3.035820e+10,3.280401e+10


In [5]:
def get_item_on_date(item, date):
    return get_item(item)[date]

In [382]:
get_item_on_date('ifrs_Equity', '20131231')

C:\Users\lyoo9\anaconda3\lib\site-packages\pandas\core\dtypes\cast.py:1229: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  v = np.array(v, copy=False)
C:\Users\lyoo9\anaconda3\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


corp_code
019660             NaN
024840    6.487671e+10
067830    2.873502e+11
101170    8.036536e+10
149950    9.793200e+10
              ...     
010140    5.845796e+12
096530    8.444518e+10
225190             NaN
196490    9.361139e+09
247540             NaN
Name: 20131231, Length: 1867, dtype: float64

In [384]:
get_item_for_period('ifrs_Equity', '20131231', '20201231', term = 'H')

C:\Users\lyoo9\anaconda3\lib\site-packages\pandas\core\dtypes\cast.py:1229: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  v = np.array(v, copy=False)
C:\Users\lyoo9\anaconda3\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


,20131231,20140630,20141231,20150630,20151231,20160630,20161231,20170630,20171231,20180630,20181231,20190630,20191231,20200630,20201231
corp_code,,,,,,,,,,,,,,,
019660,NaN,-2.153815e+08,5.852108e+09,1.031356e+10,3.784414e+10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
024840,6.487671e+10,6.267905e+10,5.817816e+10,5.503912e+10,4.313705e+10,NaN,4.605613e+10,7.551565e+10,7.938106e+10,8.580189e+10,8.262148e+10,8.635589e+10,9.330697e+10,9.685097e+10,9.594783e+10
067830,2.873502e+11,NaN,3.110314e+11,3.238736e+11,3.363721e+11,3.509564e+11,3.624767e+11,3.773556e+11,3.898040e+11,4.179906e+11,4.346336e+11,4.400014e+11,4.479321e+11,4.553963e+11,NaN
101170,8.036536e+10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.839230e+10,NaN,8.678080e+10,8.816953e+10,7.918151e+10
149950,9.793200e+10,1.010851e+11,1.126380e+11,1.135559e+11,1.195710e+11,1.175258e+11,1.202424e+11,1.250360e+11,1.375042e+11,NaN,1.370250e+11,NaN,1.360878e+11,1.350568e+11,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
010140,5.845796e+12,5.650839e+12,5.573161e+12,4.380052e+12,4.265746e+12,4.814378e+12,6.275311e+12,6.309614e+12,5.797453e+12,6.945814e+12,6.746280e+12,6.352703e+12,5.248929e+12,4.522799e+12,3.718232e+12
096530,8.444518e+10,9.013669e+10,9.483488e+10,1.125785e+11,1.353033e+11,1.381022e+11,1.200006e+11,1.423997e+11,1.172607e+11,1.479063e+11,1.283564e+11,1.410191e+11,1.528257e+11,3.098118e+11,6.464059e+11
225190,NaN,NaN,NaN,NaN,2.770598e+10,NaN,3.394844e+10,2.980370e+10,3.572604e+10,3.437798e+10,3.608955e+10,3.427206e+10,3.928479e+10,3.423793e+10,3.451267e+10


In [108]:
def get_item_for_period(item, date_from, date_to, term = 'Y'):
    # term은 Y, H, Q의 세 가지 옵션으로, 연도별, 반기별, 분기별 데이터를 얻어올 수 있도록 하는 옵션이다.
    data = []
    item_data = get_item(item)
    if term == 'Y':
        num = int(date_to[:4]) - int(date_from[:4]) +1
        date_working = date_from
        for i in range(num):
            data.append(item_data[date_working])
            date_working = date_working[:2] + str(int(date_working[2:4]) + 1) + date_working[4:]
            
    elif term == 'H':
        num = int((int(date_to[:4]) - int(date_from[:4])) * 2 + (int(date_to[4:6]) - int(date_from[4:6]))/6 + 1)
        date_working = date_from
        for i in range(num):
            try:
                data.append(item_data[date_working])
            except:
                try:
                    date_working = date_working[:6] + str(61 - int(date_working[6:]))
                    data.append(item_data[date_working])
                except KeyError:
                    print('해당 날짜의 데이터가 없습니다:', date_working)
            if int(date_working[4:6])+6 > 12:
                date_working = date_working[:2] + str(int(date_working[2:4]) + 1) +'0'+ str(int(date_working[4:6])-6)+'31'
            else:
                date_working = date_working[:4] + str(int(date_working[4:6]) + 6).zfill(2) + '31'
                
    else:
        num = int((int(date_to[:4]) - int(date_from[:4])) * 4 + (int(date_to[4:6]) - int(date_from[4:6]))/3 + 1)
        date_working = date_from
        for i in range(num):
            try:
                data.append(item_data[date_working])
            except:
                try:
                    date_working = date_working[:6] + str(61 - int(date_working[6:]))
                    data.append(item_data[date_working])
                except KeyError:
                    print('해당 날짜의 데이터가 없습니다:', date_working)
            if int(date_working[4:6])+3 > 12:
                date_working = date_working[:2] + str(int(date_working[2:4]) + 1) +'0331'
            else:
                date_working = date_working[:4] + str(int(date_working[4:6]) + 3).zfill(2) + '31'
                
    data = pd.concat(data, axis = 1)
    return data

In [96]:
get_item_for_period('ifrs_Assets', '20130331', '20180930', 'Q')

C:\Users\lyoo9\anaconda3\lib\site-packages\pandas\core\dtypes\cast.py:1229: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  v = np.array(v, copy=False)
C:\Users\lyoo9\anaconda3\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


,20130331,20130630,20130930,20131231,20140331,20140630,20140930,20141231,20150331,20150630,...,20160630,20160930,20161231,20170331,20170630,20170930,20171231,20180331,20180630,20180930
corp_code,,,,,,,,,,,,,,,,,,,,,
019660,2.886286e+10,NaN,NaN,NaN,1.703261e+10,1.055464e+10,8.087141e+09,1.010225e+10,1.280584e+10,1.315108e+10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
024840,2.269948e+11,2.316561e+11,2.147018e+11,2.054478e+11,1.897279e+11,1.892102e+11,1.850523e+11,1.712005e+11,1.846208e+11,1.968748e+11,...,NaN,NaN,1.560417e+11,1.712930e+11,1.614461e+11,2.019942e+11,1.990568e+11,1.919858e+11,1.921344e+11,1.872275e+11
067830,5.160955e+11,5.180137e+11,4.932345e+11,5.109391e+11,5.080111e+11,NaN,5.040379e+11,5.126235e+11,5.166652e+11,5.178159e+11,...,5.289845e+11,5.238916e+11,5.533600e+11,5.269696e+11,5.298662e+11,5.304646e+11,5.625620e+11,5.827113e+11,5.816357e+11,5.753262e+11
101170,1.055925e+11,1.050463e+11,1.029509e+11,9.686515e+10,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
149950,9.894424e+10,9.591215e+10,1.025936e+11,1.132099e+11,1.096481e+11,1.154352e+11,1.241637e+11,1.320943e+11,1.259007e+11,1.255599e+11,...,1.239359e+11,1.266966e+11,1.303664e+11,1.340322e+11,1.351684e+11,1.404782e+11,1.502136e+11,1.481294e+11,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
010140,1.654315e+13,1.693840e+13,1.757117e+13,1.742715e+13,1.784125e+13,1.840012e+13,1.726523e+13,1.712223e+13,1.599768e+13,1.598098e+13,...,1.749906e+13,1.647543e+13,1.721746e+13,1.543040e+13,1.494776e+13,1.359027e+13,1.381811e+13,1.407144e+13,1.423679e+13,1.399738e+13
096530,NaN,NaN,NaN,1.290390e+11,1.304947e+11,1.363690e+11,1.388936e+11,1.384533e+11,1.450635e+11,1.483856e+11,...,2.002803e+11,1.990777e+11,1.853888e+11,2.089103e+11,2.135619e+11,1.859455e+11,1.567539e+11,1.731237e+11,1.796397e+11,1.623048e+11
225190,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,4.264701e+10,NaN,3.822595e+10,4.392622e+10,4.418149e+10,4.831372e+10,4.384512e+10,4.183064e+10


In [7]:
get_item_on_date('ifrs_Assets','20131231')

C:\Users\lyoo9\anaconda3\lib\site-packages\pandas\core\dtypes\cast.py:1229: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  v = np.array(v, copy=False)
C:\Users\lyoo9\anaconda3\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


corp_code
019660             NaN
024840    2.054478e+11
067830    5.109391e+11
101170    9.686515e+10
149950    1.132099e+11
              ...     
010140    1.742715e+13
096530    1.290390e+11
225190             NaN
196490    2.239595e+10
247540             NaN
Name: 20131231, Length: 1874, dtype: float64

In [43]:
os.chdir('C:\\Users\\lyoo9\\_System Trading\\Backend\\KRX_price')
def get_market_data_on_date(date, print_error = True):
    # 에러 메시지를 출력하고 싶지 않으면 print_error를 False로 설정
    try:
        data = pd.read_csv(date[2:4] + '_' + str(int(date[4:6])) + '_' + str(int(date[6:8])) + '.csv', encoding = 'cp949')
        if data.isna()['종가'].sum() == len(data):
            raise FileNotFoundError
    except FileNotFoundError:
        if print_error:
            print('해당 날짜의 데이터가 존재하지 않습니다:', date)
        data = None
    return data

In [393]:
get_market_data_on_date('20010102')

,종목코드,종목명,시장구분,소속부,종가,대비,등락률,시가,고가,저가,거래량,거래대금,시가총액,상장주식수
0,037730,3R,KOSDAQ,NaN,20200,1850,10.08,17400,20550,17400,140850,2734839450,44375360000,2196800
1,001460,BYC,KOSPI,NaN,39550,950,2.46,39000,39600,38800,2330,91072500,24703523250,624615
2,001465,BYC(1우),KOSPI,NaN,19600,200,1.03,19900,19900,18750,600,11437000,4221546000,215385
3,035760,CJ39쇼핑,KOSDAQ,NaN,15100,1000,7.09,13900,15500,13500,69572,1031366000,124482452100,8243871
4,039720,CJ프론티어,KOSDAQ,NaN,2890,-30,-1.03,2890,2890,2890,0,0,63482029000,21966100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1506,003280,흥아해운,KOSPI,NaN,4000,100,2.56,3700,4100,3700,2120,8190000,3360000000,840000
1507,007630,흥창,KOSPI,NaN,1700,150,9.68,1510,1700,1510,415730,679403400,66639127900,39199487
1508,037440,희림,KOSDAQ,NaN,22800,-100,-0.44,20300,24600,20300,36616,841337550,22800000000,1000000
1509,000500,희성전선,KOSPI,NaN,4320,-50,-1.14,4140,4320,4140,59230,250387500,16848000000,3900000


In [237]:
def get_market_cap_on_date(date, get_code = False):
    if get_code:
        data = get_market_data_on_date(date)[['종목코드','시장구분','시가총액']].set_index('종목코드')
        data.columns = [['시장구분','Market_Cap']]
    else:
        data = get_market_data_on_date(date)[['종목코드','시가총액']].set_index('종목코드')
        data.columns = [['Market_Cap']]
    data.index.rename('corp_code', inplace = True)
    return data

In [394]:
get_market_cap_on_date('20100104', get_code = True)

,시장구분,Market_Cap
corp_code,,
069110,KOSDAQ,9209333000
060310,KOSDAQ,38016606635
900010,KOSDAQ,102720000000
013340,KOSDAQ,92460690040
054620,KOSDAQ,80497066910
...,...,...
000540,KOSPI,351008604200
000547,KOSPI,781824000
000545,KOSPI,1689600000


In [97]:
def last_market_date(date):
    # date의 기본형식은 'YYYYMM' : ex)'201212'. -> 'YYYYMMDD'도 가능하도록 코드 변경했음
    get_data = False
    day = 31
    while not get_data:
        if type(get_market_data_on_date(date[:6] + str(day), print_error = False)) != type(None):
            market_date = date[:6] + str(day)
            get_data = True
        else:
            day -=1
    return market_date

In [399]:
last_market_date('201205')

'20120531'

In [242]:
see = 0
def get_market_cap_for_period(date_from, date_to, term = 'Y', ind_match = True):
    # ind_match는 fundamental과 합치기 편하도록 실제 날짜가 아니라 월말 날짜를 표시하도록 하는 옵션
    data = []
    ind = []
    if term == 'Y':
        num = int(date_to[:4]) - int(date_from[:4]) +1
        date_working = date_from
        for i in range(num):
            data.append(get_market_cap_on_date(last_market_date(date_working), get_code = True))
            if ind_match:
                ind.append(date_working)
            else:
                ind.append(last_market_date(date_working))
            date_working = date_working[:2] + str(int(date_working[2:4]) + 1) + date_working[4:]
            
    elif term == 'H':
        num = int((int(date_to[:4]) - int(date_from[:4])) * 2 + (int(date_to[4:6]) - int(date_from[4:6]))/6 + 1)
        date_working = date_from
        for i in range(num):
            data.append(get_market_cap_on_date(last_market_date(date_working), get_code = True))
            if ind_match:
                ind.append(date_working)
            else:
                ind.append(last_market_date(date_working))
            if int(date_working[4:6])+6 > 12:
                date_working = date_working[:2] + str(int(date_working[2:4]) + 1) +'0'+ str(int(date_working[4:6])-6)+'31'
            else:
                date_working = date_working[:4] + str(int(date_working[4:6]) + 6).zfill(2) + '31'
                
    else:
        num = int((int(date_to[:4]) - int(date_from[:4])) * 4 + (int(date_to[4:6]) - int(date_from[4:6]))/3 + 1)
        date_working = date_from
        for i in range(num):
            data.append(get_market_cap_on_date(last_market_date(date_working), get_code = True))
            if ind_match:
                ind.append(date_working)
            else:
                ind.append(last_market_date(date_working))
            if int(date_working[4:6])+3 > 12:
                date_working = date_working[:2] + str(int(date_working[2:4]) + 1) +'0331'
            else:
                date_working = date_working[:4] + str(int(date_working[4:6]) + 3).zfill(2) + '31'
    cap_data = [x.iloc[:,1] for x in data]
    cat_data = [x.iloc[:,0] for x in data]
    cap_data = pd.concat(cap_data, axis =1)
    cap_data.columns = ind
    cat_data = pd.concat(cat_data, axis =1)
    cat_data.columns = ind
    return cap_data, cat_data

In [402]:
get_market_cap_for_period('20131231', '20201231', term = 'H')[1]

,20131231,20140631,20141231,20150631,20151231,20160631,20161231,20170631,20171231,20180631,20181231,20190631,20191231,20200631,20201231
060310,KOSDAQ,KOSDAQ,KOSDAQ,KOSDAQ,KOSDAQ,KOSDAQ,KOSDAQ,KOSDAQ,KOSDAQ,KOSDAQ,KOSDAQ,KOSDAQ,KOSDAQ,KOSDAQ,KOSDAQ
013340,KOSDAQ,KOSDAQ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
068400,KOSPI,KOSPI,KOSPI,KOSPI,KOSPI,KOSPI,KOSPI,KOSPI,KOSPI,KOSPI,KOSPI,KOSPI,KOSPI,KOSPI,KOSPI
006840,KOSPI,KOSPI,KOSPI,KOSPI,KOSPI,KOSPI,KOSPI,KOSPI,KOSPI,KOSPI,KOSPI,KOSPI,KOSPI,KOSPI,KOSPI
054620,KOSDAQ,KOSDAQ,KOSDAQ,KOSDAQ,KOSDAQ,KOSDAQ,KOSDAQ,KOSDAQ,KOSDAQ,KOSDAQ,KOSDAQ,KOSDAQ,KOSDAQ,KOSDAQ,KOSDAQ
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
304840,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,KOSDAQ
347770,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,KOSDAQ
343510,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,KOSDAQ
299030,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,KOSDAQ


In [404]:
get_item_for_period('ifrs_Equity', '20131231', '20201231') / get_market_cap_for_period('20131231', '20201231')[0]

,20131231,20141231,20151231,20161231,20171231,20181231,20191231,20201231
000020,NaN,NaN,NaN,1.115174,1.090378,1.166594,1.291231,0.624360
000030,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
000040,1.096480,0.370604,0.304286,0.412790,0.739080,1.083788,0.666164,0.479117
000050,2.342772,1.298043,1.256202,1.620783,1.834650,2.393804,2.914753,2.182472
000060,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
950170,NaN,NaN,NaN,NaN,NaN,0.101500,0.083704,0.091300
950180,NaN,NaN,NaN,NaN,NaN,NaN,0.082403,0.048702
950190,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.002259
950200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [251]:
def get_market_category(data):
    market_cat = []
    for i in range(len(data.columns)):
        market_cat.append(get_market_cap_on_date(last_market_date(data.columns[i]), get_code = True).iloc[:,0])
    market_cat = pd.concat(market_cat, axis=1)
    market_cat.columns = data.columns
    return market_cat

In [405]:
get_market_category(equity)

,20131231,20141231,20151231,20161231,20171231,20181231,20191231,20201231
060310,KOSDAQ,KOSDAQ,KOSDAQ,KOSDAQ,KOSDAQ,KOSDAQ,KOSDAQ,KOSDAQ
013340,KOSDAQ,NaN,NaN,NaN,NaN,NaN,NaN,NaN
068400,KOSPI,KOSPI,KOSPI,KOSPI,KOSPI,KOSPI,KOSPI,KOSPI
006840,KOSPI,KOSPI,KOSPI,KOSPI,KOSPI,KOSPI,KOSPI,KOSPI
054620,KOSDAQ,KOSDAQ,KOSDAQ,KOSDAQ,KOSDAQ,KOSDAQ,KOSDAQ,KOSDAQ
...,...,...,...,...,...,...,...,...
347770,NaN,NaN,NaN,NaN,NaN,NaN,NaN,KOSDAQ
341160,NaN,NaN,NaN,NaN,NaN,NaN,NaN,KOSDAQ
343510,NaN,NaN,NaN,NaN,NaN,NaN,NaN,KOSDAQ
299030,NaN,NaN,NaN,NaN,NaN,NaN,NaN,KOSDAQ


In [303]:
def get_breakpoint(data, breakpoint):
    absolute_point = []
    rank = data.rank(pct=True)
    for i in range(len(breakpoint)):
        absolute_point.append((data[rank < breakpoint[i]].max() + data[rank > breakpoint[i]].min())/2)
    return absolute_point

In [313]:
def get_mask(data, market_cat, market_bp = 'All', breakpoint: list = [0.5]):
    # data는 mask의 기준이 되는 baseline data
    # market_bp는 'ALL', 'KOSPI', 'KOSDAQ'으로, 기준이 되는 breakpoint의 market을 의미
    # breakpoint는 masking의 구분점으로, list안에 float이 담긴 형태를 받음
    breakpoint = np.sort(breakpoint)
    mask_list = []
    for i in range(len(data.columns)):
        mask = data.iloc[:,i] * 0 +1
        mask_np = -np.ones(len(mask))
        #rank = data.iloc[:,i].rank(pct = True)
        if market_bp == 'All':
            absolute_point = get_breakpoint(data.iloc[:,i], breakpoint)
        if market_bp == 'KOSPI':
            absolute_point = get_breakpoint(data[market_cat.iloc[:,i]=='KOSPI'].iloc[:,i], breakpoint)
        if market_bp == 'KOSDAQ':
            absolute_point = get_breakpoint(data[market_cat.iloc[:,i]=='KOSDAQ'].iloc[:,i], breakpoint)
        for j in range(len(breakpoint)+1):
            if j < len(breakpoint):
                for k in range(len(mask)):
                    mask_np[k] = j if data.iloc[k,i] < absolute_point[j] and mask_np[k] == -1 else mask_np[k]
            else:
                for k in range(len(mask)):
                    mask_np[k] = j if data.iloc[k,i] > absolute_point[j-1] and mask_np[k] == -1 else mask_np[k]
        mask = mask * mask_np
        mask_list.append(mask)
    mask_df = pd.concat(mask_list, axis =1)
    return mask_df

In [314]:
equity = get_item_for_period('ifrs_Equity', '20131231', '20201231')

C:\Users\lyoo9\anaconda3\lib\site-packages\pandas\core\dtypes\cast.py:1229: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  v = np.array(v, copy=False)
C:\Users\lyoo9\anaconda3\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [430]:
get_mask(equity, get_market_category(equity), breakpoint = [0.2, 0.8])

,20131231,20141231,20151231,20161231,20171231,20181231,20191231,20201231
corp_code,,,,,,,,
019660,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN
024840,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0
067830,2.0,2.0,2.0,2.0,2.0,2.0,2.0,NaN
101170,1.0,NaN,NaN,NaN,NaN,1.0,1.0,1.0
149950,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN
...,...,...,...,...,...,...,...,...
010140,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
096530,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0
225190,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0


In [426]:
get_mask(get_market_cap_for_period('20130630', '20200630')[0], get_market_cap_for_period('20130630', '20200630')[1])

,20130630,20140630,20150630,20160630,20170630,20180630,20190630,20200630
060310,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0
013340,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
068400,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
006840,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
054620,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...
318000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
256150,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
272210,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
340440,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0


In [ ]:
def shift_date_quarter(data, num_of_quarters):
    # data의 date를 원하는 만큼 밀어줌.
    dates = data.columns
    years = [int(x[:4]) for x in dates]
    months = [int(x[4:6]) for x in dates]
    shifted_months = [x + num_of_quarters * 3 for x in dates]
    shifted_years = [x + (shifted_months[i]-1) // 12 for x, i enumerate(years)]
    dates = [x[:4] + str(int(x[4:6]) + num_of_quarters * 3) + x[6:] for x in dates]
    dates = [ for x in dates]
    

In [ ]:
def get_factor_on_date_by_mask(mask):
    # 12 -> 3 은 날짜가 다른 날들보다 조금 적은 것 같아서 안전하게 일주일로 잡았음. 
    # @@@@@@@@@@@ EW 말고 VW도 추가하면 좋을 듯. 혹은 두 개 combination도 나중에는 고려해봐야할듯.
    # @@@@@@@@@@@ option 추가해야할듯! : daily / weekly / monthly / quarterly 정도까지는 구현을 해놓아야할듯
    term_conservative = pd.Timestamp(mask.columns[1]) - pd.Timestamp(mask.columns[0]) + pd.Timedelta(days = 7)
    date_to = pd.Timestamp(mask.columns[-1]) + term_conservative
    date_from = pd.Timestamp(mask.columns[0]) + pd.Timedelta(days = 1)
    factor_days = (date_to - date_from).days +1
    date_point = date_from
    for i in range(factor_days):
        get_market_data_on_date(date_point.strftime('%Y%m%d'))

In [378]:
type((pd.Timestamp('20121231') - pd.Timestamp('20120930') + pd.Timedelta(days = 7)).days)

int

In [ ]:
pfo에다가 돌려보기! factor
